上二つのコードセルを一度実行してからランタイムを再起動する

In [ ]:
!curl -L https://raw.githubusercontent.com/facebookresearch/habitat-sim/main/examples/colab_utils/colab_install.sh | NIGHTLY=true bash -s
!wget -c http://dl.fbaipublicfiles.com/habitat/mp3d_example.zip && unzip -o mp3d_example.zip -d /content/habitat-sim/data/scene_datasets/mp3d/
#!wget -c https://dl.fbaipublicfiles.com/habitat/data/datasets/pointnav/gibson/v1/pointnav_gibson_v1.zip && unzip -o pointnav_gibson_v1.zip -d /content/habitat-sim/data/datasets/pointnav/gibson/v1/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2574  100  2574    0     0  14542      0 --:--:-- --:--:-- --:--:-- 14542
--2022-04-20 11:22:26--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-20 11:22:26--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response..

In [ ]:
!pip uninstall --yes pyopenssl
!pip install pyopenssl

Found existing installation: pyOpenSSL 22.0.0
Uninstalling pyOpenSSL-22.0.0:
  Successfully uninstalled pyOpenSSL-22.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 2.2 MB/s eta 0:00:00


**ここで一度ランタイムを再起動。**
(PIL.Image.Resamplingが使えないエラーを回避できる)
再起動後はこの下から実行

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/habitat-lab

/content/habitat-lab


In [ ]:
#!/usr/bin/env python3

# Copyright (c) Facebook, Inc. and its affiliates.
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

import argparse
import random

import numpy as np
import torch

from habitat.config import Config
from habitat_baselines.common.baseline_registry import baseline_registry
from habitat_baselines.config.default import get_config

In [ ]:
def train(path):

    config = get_config("habitat_baselines/config/pointnav/ppo_pointnav_example.yaml")
    
    config["TOTAL_NUM_STEPS"] = -1
    config["NUM_UPDATES"] = 500000
    config["NUM_CHECKPOINTS"] = -1
    config["CHECKPOINT_INTERVAL"] = 1000

    config["CHECKPOINT_FOLDER"] = config["EVAL_CKPT_PATH_DIR"] = path + "/new_checkpoints"
    config["VIDEO_DIR"] = path + "/video_dir"

    random.seed(config.TASK_CONFIG.SEED)
    np.random.seed(config.TASK_CONFIG.SEED)
    torch.manual_seed(config.TASK_CONFIG.SEED)
    if config.FORCE_TORCH_SINGLE_THREADED and torch.cuda.is_available():
        torch.set_num_threads(1)

    trainer_init = baseline_registry.get_trainer(config.TRAINER_NAME)
    assert trainer_init is not None, f"{config.TRAINER_NAME} is not supported"
    trainer = trainer_init(config)

    trainer.train()

train("/content/drive/MyDrive/habitat_on_colab/exps/new_folder")

ストリーミング出力は最後の 5000 行に切り捨てられました。
2022-04-20 08:13:05,983 update: 62800	env-time: 8561.944s	pth-time: 12743.873s	frames: 2009600
2022-04-20 08:13:05,985 Average window size: 50  distance_to_goal: 4.528  reward: 4.591  spl: 0.348  success: 0.441
2022-04-20 08:13:09,653 update: 62810	fps: 90.649	
2022-04-20 08:13:09,654 update: 62810	env-time: 8563.417s	pth-time: 12745.972s	frames: 2009920
2022-04-20 08:13:09,660 Average window size: 50  distance_to_goal: 3.472  reward: 6.265  spl: 0.468  success: 0.567
2022-04-20 08:13:13,224 update: 62820	fps: 90.648	
2022-04-20 08:13:13,226 update: 62820	env-time: 8564.851s	pth-time: 12748.002s	frames: 2010240
2022-04-20 08:13:13,233 Average window size: 50  distance_to_goal: 2.163  reward: 7.576  spl: 0.580  success: 0.692
2022-04-20 08:13:17,124 update: 62830	fps: 90.647	
2022-04-20 08:13:17,126 update: 62830	env-time: 8566.382s	pth-time: 12750.254s	frames: 2010560
2022-04-20 08:13:17,128 Average window size: 50  distance_to_goal: 1.194  reward: 9.440

In [ ]:
def eval(path):

    config = get_config("habitat_baselines/config/pointnav/ppo_pointnav_example.yaml")
    
    config["TOTAL_NUM_STEPS"] = -1
    config["NUM_UPDATES"] = 500000
    config["NUM_CHECKPOINTS"] = -1
    config["CHECKPOINT_INTERVAL"] = 1000
    config["CHECKPOINT_FOLDER"] = config["EVAL_CKPT_PATH_DIR"] = path + "/new_checkpoints"
    config["VIDEO_DIR"] = path + "/video_dir"

    random.seed(config.TASK_CONFIG.SEED)
    np.random.seed(config.TASK_CONFIG.SEED)
    torch.manual_seed(config.TASK_CONFIG.SEED)
    if config.FORCE_TORCH_SINGLE_THREADED and torch.cuda.is_available():
        torch.set_num_threads(1)

    trainer_init = baseline_registry.get_trainer(config.TRAINER_NAME)
    assert trainer_init is not None, f"{config.TRAINER_NAME} is not supported"
    trainer = trainer_init(config)

    trainer.eval()

eval("/content/drive/MyDrive/habitat_on_colab/exps/new_folder")

ストリーミング出力は最後の 5000 行に切り捨てられました。
      DIST_REWARD: 1.0
      FORCE_END_PEN: 10.0
      FORCE_PEN: 0.0
      GRASP_REWARD: 0.0
      MARKER_DIST_REWARD: 0.0
      MAX_FORCE_PEN: 1.0
      TYPE: ArtObjReward
      WRONG_GRASP_END: True
      WRONG_GRASP_PEN: 5.0
    ART_OBJ_STATE:
      TYPE: ArtObjState
    ART_OBJ_SUCCESS:
      SUCCESS_DIST_THRESHOLD: 0.05
      TYPE: ArtObjSuccess
      USE_ABSOLUTE_DISTANCE: True
    ART_SUCC_THRESH: 0.15
    BAD_CALLED_TERMINATE:
      BAD_TERM_PEN: 0.0
      DECAY_BAD_TERM: False
      TYPE: BadCalledTerminate
    BASE_ANGLE_NOISE: 0.15
    BASE_NOISE: 0.05
    COLLISIONS:
      TYPE: Collisions
    COMPASS_SENSOR:
      TYPE: CompassSensor
    COMPOSITE_NODE_IDX:
      TYPE: CompositeNodeIdx
    COMPOSITE_REWARD:
      STAGE_COMPLETE_REWARD: 10.0
      SUCCESS_REWARD: 10.0
      TYPE: CompositeReward
    COMPOSITE_SUCCESS:
      TYPE: CompositeSuccess
    CONSTRAINT_VIOLATE_SHOULD_END: False
    CONSTRAINT_VIOLATION_ENDS_EPISODE: False
    CORREC

KeyboardInterrupt: ignored

In [ ]:
!cp /content/habitat-lab/tb/events.out.tfevents.1650454264.94a4520e1594.3039.0 /content/drive/MyDrive/habitat_on_colab/exps/new_folder